## Prepare data ##
Cell 1: Configuration
AWESOME_CV_BASE, 
OUTPUT_ORGANIZED 

Cell 2: Run organization
main()

Cell 3: Explore specific category
from pathlib import Path

List all summary files
summary_dir = Path(OUTPUT_ORGANIZED) / "summary"
for f in sorted(summary_dir.glob("*.tex"))[:10]:
    print(f.name)

Cell 4: Search for specific content
analyzer = CVAnalysisTools(OUTPUT_ORGANIZED)

Find all cancer-related experience files
cancer_files = analyzer.find_similar_files("cancer", category="experience")

Cell 5: Compare two similar files
if len(cancer_files) >= 2:
    analyzer.compare_files(cancer_files[0], cancer_files[1])
```

📂 Output Structure
```
organized_files/
├── summary/
│   ├── Bioinformatics_2025_03__summary_bioinformatics_busi_2025_03.tex
│   ├── DS_2025__summary_DS_project_busi_2025_03.tex
│   └── ... (many more)
├── experience/
│   ├── Bioinformatics_2025_03__experience_bioinformatics_2025_03.tex
│   ├── DS_Healthcare_2025_05__experience_DS_2025_05.tex
│   └── ...
├── key_skills/
│   ├── Bioinformatics_2025_05__key_skills_2025_05.tex
│   ├── IT_HPC_2025_04__key_skills_2025_04_IT.tex
│   └── ...
├── education/
├── additional_info/
├── file_mappings.json
├── ORGANIZATION_REPORT.md
└── statistics.csv

In [ ]:
# ═══════════════════════════════════════════════════════════════════
# 📁 CV FILES ORGANIZER
# Extract and organize CV component files from Awesome-CV directories
# Groups files by type (summary, skills, experience, etc.)
# ═══════════════════════════════════════════════════════════════════

import os
import re
import shutil
from pathlib import Path
from collections import defaultdict
from typing import Dict, List, Set, Tuple
import pandas as pd

# ═══════════════════════════════════════════════════════════════════
# CONFIGURATION
# ═══════════════════════════════════════════════════════════════════

# Update these paths
AWESOME_CV_BASE = "/Users/kbillis/bin/Awesome-CV/examples"
OUTPUT_ORGANIZED = "/Users/kbillis/bin/Awesome-CV/organized_files"

# ═══════════════════════════════════════════════════════════════════
# CV FILES EXTRACTOR
# ═══════════════════════════════════════════════════════════════════

class CVFilesOrganizer:
    """Extract and organize CV component files."""
    
    def __init__(self, base_path: str, output_path: str):
        self.base_path = Path(base_path)
        self.output_path = Path(output_path)
        self.file_mapping = defaultdict(list)
        
        # File type categories
        self.file_categories = {
            'summary': ['summary', 'objective'],
            'experience': ['experience', 'work', 'employment'],
            'key_skills': ['key_skills', 'skills', 'competencies'],
            'education': ['education', 'academic'],
            'additional_info': ['additional_info', 'additional', 'extra'],
            'honors': ['honors', 'awards', 'achievements'],
            'publications': ['publications', 'papers', 'writing'],
            'projects': ['projects', 'portfolio']
        }
    
    def find_all_resume_files(self) -> List[Path]:
        """Find all resume*.tex files."""
        resume_files = []
        
        print(f"🔍 Scanning {self.base_path} for resume files...")
        
        for resume_file in self.base_path.rglob("resume*.tex"):
            if resume_file.is_file():
                resume_files.append(resume_file)
        
        print(f"✅ Found {len(resume_files)} resume files")
        return resume_files
    
    def extract_input_files(self, resume_file: Path) -> List[Tuple[str, Path]]:
        """
        Extract all \input{...} file references from a resume file.
        Returns list of (category, filepath) tuples.
        Only includes non-commented lines.
        """
        input_files = []
        
        try:
            with open(resume_file, 'r', encoding='utf-8', errors='ignore') as f:
                content = f.read()
            
            # Find all \input{...} commands that are NOT commented
            lines = content.split('\n')
            
            for line in lines:
                # Skip commented lines
                if line.strip().startswith('%'):
                    continue
                
                # Find \input{filepath} pattern
                match = re.search(r'\\input\{(.+?)\}', line)
                if match:
                    filepath_str = match.group(1)
                    
                    # Handle both absolute and relative paths
                    if filepath_str.startswith('/'):
                        filepath = Path(filepath_str)
                    else:
                        filepath = resume_file.parent / filepath_str
                    
                    # Only include if file exists
                    if filepath.exists() and filepath.is_file():
                        # Determine category
                        category = self._categorize_file(filepath.name)
                        input_files.append((category, filepath))
        
        except Exception as e:
            print(f"   ⚠️  Error reading {resume_file}: {e}")
        
        return input_files
    
    def _categorize_file(self, filename: str) -> str:
        """Categorize a file based on its name."""
        filename_lower = filename.lower()
        
        for category, keywords in self.file_categories.items():
            for keyword in keywords:
                if keyword in filename_lower:
                    return category
        
        return 'other'
    
    def organize_files(self):
        """Main function to organize all CV files."""
        print("\n" + "="*70)
        print("📁 CV FILES ORGANIZER")
        print("="*70)
        
        # Find all resume files
        resume_files = self.find_all_resume_files()
        
        if not resume_files:
            print("❌ No resume files found!")
            return
        
        # Extract input files from each resume
        print(f"\n📋 Extracting input files from resumes...")
        
        all_input_files = defaultdict(set)
        resume_mappings = []
        
        for resume_file in resume_files:
            print(f"   Processing: {resume_file.parent.name}/{resume_file.name}")
            
            input_files = self.extract_input_files(resume_file)
            
            mapping_entry = {
                'resume': str(resume_file),
                'directory': resume_file.parent.name,
                'files': []
            }
            
            for category, filepath in input_files:
                all_input_files[category].add(filepath)
                mapping_entry['files'].append({
                    'category': category,
                    'path': str(filepath),
                    'name': filepath.name
                })
            
            resume_mappings.append(mapping_entry)
            
            if input_files:
                print(f"      ✓ Found {len(input_files)} input files")
        
        # Create organized directory structure
        print(f"\n📂 Creating organized directory structure...")
        self.output_path.mkdir(parents=True, exist_ok=True)
        
        stats = {}
        
        for category, filepaths in all_input_files.items():
            if not filepaths:
                continue
            
            category_dir = self.output_path / category
            category_dir.mkdir(exist_ok=True)
            
            print(f"\n   📁 {category.upper()}: {len(filepaths)} unique files")
            
            copied_count = 0
            for filepath in filepaths:
                try:
                    # Create unique filename to avoid conflicts
                    # Format: original_directory__filename.tex
                    source_dir = filepath.parent.name
                    new_filename = f"{source_dir}__{filepath.name}"
                    dest_path = category_dir / new_filename
                    
                    # Copy file
                    shutil.copy2(filepath, dest_path)
                    copied_count += 1
                    
                except Exception as e:
                    print(f"      ⚠️  Error copying {filepath.name}: {e}")
            
            stats[category] = {
                'total': len(filepaths),
                'copied': copied_count
            }
            
            print(f"      ✅ Copied {copied_count}/{len(filepaths)} files")
        
        # Create index/mapping file
        print(f"\n📄 Creating mapping file...")
        self._create_mapping_file(resume_mappings)
        
        # Create summary report
        self._create_summary_report(stats, resume_mappings)
        
        print(f"\n✅ Organization complete!")
        print(f"   Output directory: {self.output_path}")
        print(f"   Total categories: {len(stats)}")
        print(f"   Total unique files: {sum(s['copied'] for s in stats.values())}")
    
    def _create_mapping_file(self, resume_mappings: List[Dict]):
        """Create a JSON mapping file showing which files belong to which resume."""
        import json
        
        mapping_file = self.output_path / "file_mappings.json"
        
        with open(mapping_file, 'w') as f:
            json.dump(resume_mappings, f, indent=2)
        
        print(f"      ✅ Saved: file_mappings.json")
    
    def _create_summary_report(self, stats: Dict, resume_mappings: List[Dict]):
        """Create a summary report in markdown format."""
        
        report_file = self.output_path / "ORGANIZATION_REPORT.md"
        
        with open(report_file, 'w') as f:
            f.write("# CV Files Organization Report\n\n")
            f.write(f"Generated: {pd.Timestamp.now()}\n\n")
            
            f.write("## Summary\n\n")
            f.write(f"- Total resumes processed: {len(resume_mappings)}\n")
            f.write(f"- Total categories: {len(stats)}\n")
            f.write(f"- Total unique files: {sum(s['copied'] for s in stats.values())}\n\n")
            
            f.write("## Files by Category\n\n")
            for category, stat in sorted(stats.items()):
                f.write(f"### {category.replace('_', ' ').title()}\n")
                f.write(f"- Files: {stat['copied']}\n")
                f.write(f"- Location: `{category}/`\n\n")
            
            f.write("## Directory Structure\n\n")
            f.write("```\n")
            f.write("organized_files/\n")
            for category in sorted(stats.keys()):
                f.write(f"├── {category}/\n")
                f.write(f"│   └── (Contains {stats[category]['copied']} files)\n")
            f.write("├── file_mappings.json\n")
            f.write("└── ORGANIZATION_REPORT.md\n")
            f.write("```\n\n")
            
            f.write("## File Naming Convention\n\n")
            f.write("Files are named as: `{original_directory}__{original_filename}.tex`\n\n")
            f.write("This preserves the source directory information and prevents filename conflicts.\n\n")
            
            f.write("## Usage\n\n")
            f.write("1. Browse files by category in their respective directories\n")
            f.write("2. Use `file_mappings.json` to see which files belong to which resume\n")
            f.write("3. Search for specific patterns across categories\n")
            f.write("4. Use as reference for creating new CVs\n")
        
        print(f"      ✅ Saved: ORGANIZATION_REPORT.md")
    
    def create_category_index(self):
        """Create an index file for each category showing all files."""
        print(f"\n📋 Creating category indexes...")
        
        for category_dir in self.output_path.iterdir():
            if not category_dir.is_dir():
                continue
            
            category = category_dir.name
            index_file = category_dir / "INDEX.md"
            
            # Get all files in this category
            tex_files = sorted(category_dir.glob("*.tex"))
            
            with open(index_file, 'w') as f:
                f.write(f"# {category.replace('_', ' ').title()} Files\n\n")
                f.write(f"Total files: {len(tex_files)}\n\n")
                
                f.write("## Files\n\n")
                for tex_file in tex_files:
                    # Extract original directory name
                    parts = tex_file.stem.split('__')
                    original_dir = parts[0] if len(parts) > 1 else 'unknown'
                    original_name = '__'.join(parts[1:]) if len(parts) > 1 else tex_file.stem
                    
                    f.write(f"### {tex_file.name}\n")
                    f.write(f"- **Source Directory:** {original_dir}\n")
                    f.write(f"- **Original Name:** {original_name}.tex\n")
                    f.write(f"- **Size:** {tex_file.stat().st_size} bytes\n")
                    
                    # Try to read first few lines
                    try:
                        with open(tex_file, 'r', encoding='utf-8') as tf:
                            lines = [line.strip() for line in tf.readlines()[:5] if line.strip() and not line.strip().startswith('%')]
                            if lines:
                                f.write(f"- **Preview:**\n```latex\n{chr(10).join(lines[:3])}\n...\n```\n")
                    except:
                        pass
                    
                    f.write("\n")
            
            print(f"   ✅ Created index for: {category}")

# ═══════════════════════════════════════════════════════════════════
# ANALYSIS TOOLS
# ═══════════════════════════════════════════════════════════════════

class CVAnalysisTools:
    """Additional analysis tools for organized CV files."""
    
    def __init__(self, organized_path: str):
        self.organized_path = Path(organized_path)
    
    def find_similar_files(self, query: str, category: str = None) -> List[Path]:
        """Find files matching a query string."""
        print(f"\n🔍 Searching for: '{query}'")
        
        if category:
            search_dirs = [self.organized_path / category]
        else:
            search_dirs = [d for d in self.organized_path.iterdir() if d.is_dir()]
        
        matching_files = []
        
        for search_dir in search_dirs:
            if not search_dir.exists():
                continue
            
            for tex_file in search_dir.glob("*.tex"):
                try:
                    with open(tex_file, 'r', encoding='utf-8', errors='ignore') as f:
                        content = f.read().lower()
                        if query.lower() in content:
                            matching_files.append(tex_file)
                except:
                    pass
        
        print(f"✅ Found {len(matching_files)} matching files")
        
        return matching_files
    
    def compare_files(self, file1: Path, file2: Path):
        """Compare two CV files and show differences."""
        print(f"\n📊 Comparing:")
        print(f"   File 1: {file1.name}")
        print(f"   File 2: {file2.name}")
        
        try:
            with open(file1, 'r') as f:
                content1 = set(f.readlines())
            
            with open(file2, 'r') as f:
                content2 = set(f.readlines())
            
            unique_to_1 = content1 - content2
            unique_to_2 = content2 - content1
            common = content1 & content2
            
            print(f"\n   Common lines: {len(common)}")
            print(f"   Unique to {file1.name}: {len(unique_to_1)}")
            print(f"   Unique to {file2.name}: {len(unique_to_2)}")
            
        except Exception as e:
            print(f"   ❌ Error: {e}")
    
    def create_statistics_report(self):
        """Create detailed statistics about organized files."""
        print(f"\n📊 Generating statistics...")
        
        stats = {}
        
        for category_dir in self.organized_path.iterdir():
            if not category_dir.is_dir():
                continue
            
            category = category_dir.name
            tex_files = list(category_dir.glob("*.tex"))
            
            # Calculate statistics
            total_size = sum(f.stat().st_size for f in tex_files)
            avg_size = total_size / len(tex_files) if tex_files else 0
            
            # Count unique source directories
            source_dirs = set()
            for tex_file in tex_files:
                parts = tex_file.stem.split('__')
                if len(parts) > 1:
                    source_dirs.add(parts[0])
            
            stats[category] = {
                'files': len(tex_files),
                'total_size': total_size,
                'avg_size': avg_size,
                'source_dirs': len(source_dirs)
            }
        
        # Create DataFrame
        df = pd.DataFrame(stats).T
        df = df.sort_values('files', ascending=False)
        
        print("\n" + "="*70)
        print("📈 STATISTICS SUMMARY")
        print("="*70)
        print(df.to_string())
        
        # Save to CSV
        stats_file = self.organized_path / "statistics.csv"
        df.to_csv(stats_file)
        print(f"\n✅ Saved statistics to: {stats_file}")

# ═══════════════════════════════════════════════════════════════════
# MAIN EXECUTION
# ═══════════════════════════════════════════════════════════════════

def main():
    """Main execution function."""
    
    print("="*70)
    print("📁 CV FILES ORGANIZER")
    print("="*70)
    print(f"\n📂 Source: {AWESOME_CV_BASE}")
    print(f"📂 Output: {OUTPUT_ORGANIZED}")
    
    # Create organizer
    organizer = CVFilesOrganizer(AWESOME_CV_BASE, OUTPUT_ORGANIZED)
    
    # Organize files
    organizer.organize_files()
    
    # Create category indexes
    organizer.create_category_index()
    
    # Run analysis
    print(f"\n" + "="*70)
    print("📊 RUNNING ANALYSIS")
    print("="*70)
    
    analyzer = CVAnalysisTools(OUTPUT_ORGANIZED)
    analyzer.create_statistics_report()
    
    # Example searches
    print(f"\n" + "="*70)
    print("🔍 EXAMPLE SEARCHES")
    print("="*70)
    
    # Search for bioinformatics-related summaries
    bio_files = analyzer.find_similar_files("bioinformatics", category="summary")
    if bio_files:
        print(f"\n   Top bioinformatics summaries:")
        for f in bio_files[:3]:
            print(f"      • {f.name}")
    
    # Search for Python skills
    python_files = analyzer.find_similar_files("python", category="key_skills")
    if python_files:
        print(f"\n   Files mentioning Python:")
        for f in python_files[:3]:
            print(f"      • {f.name}")
    
    print("\n" + "="*70)
    print("✅ ORGANIZATION COMPLETE!")
    print("="*70)
    print(f"\n📁 Check output directory: {OUTPUT_ORGANIZED}")
    print(f"📄 Read: ORGANIZATION_REPORT.md")
    print(f"📋 Browse categories:")
    
    for category_dir in Path(OUTPUT_ORGANIZED).iterdir():
        if category_dir.is_dir():
            file_count = len(list(category_dir.glob("*.tex")))
            print(f"      • {category_dir.name}/  ({file_count} files)")

if __name__ == "__main__":
    main()

<>:62: SyntaxWarning: invalid escape sequence '\i'
<>:62: SyntaxWarning: invalid escape sequence '\i'
/var/folders/8l/dg2tx1j914nbpcy558d8wzym0000gn/T/ipykernel_63680/117704964.py:62: SyntaxWarning: invalid escape sequence '\i'
  Extract all \input{...} file references from a resume file.


📁 CV FILES ORGANIZER

📂 Source: /Users/kbillis/bin/Awesome-CV/examples
📂 Output: /Users/kbillis/bin/Awesome-CV/organized_files

📁 CV FILES ORGANIZER
🔍 Scanning /Users/kbillis/bin/Awesome-CV/examples for resume files...
✅ Found 83 resume files

📋 Extracting input files from resumes...
   Processing: client_Platform_Nvidia_Med_2025_10/resume_konstantinos_billis.tex
      ✓ Found 5 input files
   Processing: Bioinformatics_2025_09_biomarkers/resume_konstantinos_billis_25_09_Biomarkers.tex
      ✓ Found 5 input files
   Processing: Software_dev_2025_09/resume_konstantinos_billis_25_09_softw.tex
      ✓ Found 5 input files
   Processing: client_Platform_Med_2025_10/resume_konstantinos_billis.tex
      ✓ Found 5 input files
   Processing: Data_Architect_2025_06/resume_konstantinos_billis.tex
      ✓ Found 4 input files
   Processing: DS_engineer_2024_03/resume_konstantinos_billis_25_03_DS.tex
      ✓ Found 4 input files
   Processing: Data_Architect_2025_09/resume_konstantinos_billis.tex
   

In [5]:
import os
import hashlib
import shutil
from collections import defaultdict

def hash_file(filepath, chunk_size=8192):
    """Generate SHA256 hash for a file."""
    hasher = hashlib.sha256()
    with open(filepath, 'rb') as f:
        while chunk := f.read(chunk_size):
            hasher.update(chunk)
    return hasher.hexdigest()

def copy_unique_files(src_dir, dest_dir):
    """Copy only unique files from src_dir to dest_dir."""
    os.makedirs(dest_dir, exist_ok=True)
    seen_hashes = set()
    hash_to_path = {}

    for dirpath, _, filenames in os.walk(src_dir):
        for filename in filenames:
            full_path = os.path.join(dirpath, filename)
            try:
                file_hash = hash_file(full_path)
                if file_hash not in seen_hashes:
                    seen_hashes.add(file_hash)
                    hash_to_path[file_hash] = full_path

                    # Preserve relative path structure
                    rel_path = os.path.relpath(full_path, src_dir)
                    dest_path = os.path.join(dest_dir, rel_path)
                    os.makedirs(os.path.dirname(dest_path), exist_ok=True)
                    shutil.copy2(full_path, dest_path)
                    print(f"✅ Copied: {rel_path}")
                else:
                    print(f"🔁 Duplicate skipped: {filename}")
            except Exception as e:
                print(f"⚠️ Error processing {full_path}: {e}")

    return hash_to_path

# 🔍 Run the deduplication
source_directory = OUTPUT_ORGANIZED
output_directory = OUTPUT_ORGANIZED+"deduplicated/"
unique_files = copy_unique_files(source_directory, output_directory)

print(f"\n✅ Finished. {len(unique_files)} unique files copied to {output_directory}")


✅ Copied: file_mappings.json
✅ Copied: ORGANIZATION_REPORT.md
✅ Copied: statistics.csv
✅ Copied: experience/business_Data_Prec_Med_2025_10__experience_bioinformatics.tex
✅ Copied: experience/Bioinformatics_2025_08__experience_bioinformatics.tex
✅ Copied: experience/DS_engineer_2024_03__experience_DS_it_2025_03.tex
✅ Copied: experience/Bioinformatics_Data_product_2025_04__experience_data_product_2025_04.tex
✅ Copied: experience/DS_Healthcare_2025_05__experience_DS_2025_05.tex
✅ Copied: experience/Bioinformatics_2025_07_biot__experience_bioinformatics.tex
✅ Copied: experience/Data_Science_Novartis_2025_08__experience_bioinformatics.tex
✅ Copied: experience/LEAD_data_engineer_JJ_2025_10__experience_bioinformatics.tex
✅ Copied: experience/IT_HPC_Pfizer_2025_04__experience_IT_2025_04.tex
✅ Copied: experience/Bioinformatics_2025_09_biomarkers__experience_bioinformatics.tex
✅ Copied: experience/Plants_2025_03__experience_bioinformatics_2025_03.tex
✅ Copied: experience/Senior_Bioinformatics_Sc

✅ Copied: key_skills/client_Platform_Med_2025_10__key_skills.tex
✅ Copied: key_skills/Plants_2025_09__key_skills_2025_03.tex
✅ Copied: summary/Bioinformatics_2025_09_cancer__summary_bioinformatics.tex
✅ Copied: summary/business_Data_Med_2025_09__summary_bioinformatics.tex
✅ Copied: summary/Bioinformatics_2025_03__summary_bioinformatics_busi_2025_03.tex
✅ Copied: summary/DS_Healthcare_2025_09_patient_data__summary_bioinformatics_DS_business_2025_03.tex
✅ Copied: summary/business_Data_Prec_Med_2025_10__summary_bioinformatics.tex
✅ Copied: summary/DS_2025__summary_DS_project_busi_2025_03.tex
✅ Copied: summary/Data_Science_Novartis_2025_08__summary_bioinformatics.tex
✅ Copied: summary/Data_Architect_2025_07__summary_data_busi.tex
✅ Copied: summary/Bioinformatics_2024_09__summary_bioinformatics_busi_2024_09.tex
✅ Copied: summary/Bioinformatics_2024_09__summary_bioinformatics_2024_09.tex
✅ Copied: summary/Bioinformatics_2025_07_biot__summary_bioinformatics.tex
✅ Copied: summary/Data_Science_